### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]
 [ 7.  8. 10.  8.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]
 [ 9. 10. 10.  9.]
 [10.  9. 10.  8.]
 [ 5.  8.  5.  8.]
 [10.  8. 10. 10.]
 [ 9.  9. 10. 10.]
 [ 9.  8.  8.  8.]
 [10.  8.  1. 10.]
 [ 5.  6. 10. 10.]
 [ 8.  7. 10.  7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [23]:
user_movie_subset.iloc[0,0]# == np.nan
# n_users=3
# latent_features=4
# user_mat = np.random.rand(n_users,latent_features)
# user_mat[0]

10.0

In [55]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]# number of rows in the matrix
    n_movies = ratings_mat.shape[1]# number of movies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))#n_users*n_movies# total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users,latent_features)# user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies)# movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iter in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
#                 print(i)
#                 print(j)
#                 print(ratings_mat.iloc[i,j])
                # if the rating exists
                if ratings_mat[i,j] > 0: # np.nan:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    pred_rat = np.dot(user_mat[i,:], movie_mat[:,j])
                    error = ratings_mat[i,j] - pred_rat

                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += error ** 2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i,k] = user_mat[i,k] + learning_rate * 2* (error * movie_mat[k,j])
                        movie_mat[k,j] = movie_mat[k,j] + learning_rate * 2* (error * user_mat[i,k])

        # print results for iteration
        print("%d \t\t %f" % (iter+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [46]:
user_mat, movie_mat = FunkSVD(user_movie_subset, latent_features=4, learning_rate=0.005, iters=10)
# use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 45.797522
2 		 15.996829
3 		 4.215083
4 		 2.862046
5 		 2.758651
6 		 2.729819
7 		 2.709324
8 		 2.688853
9 		 2.666412
10 		 2.641244


In [47]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[10.06079888  8.84341245 10.73055434 10.08783556]
 [ 8.66514539  7.61468803  9.09924406  8.64000346]
 [ 7.73896561  6.8943657   8.43995065  7.71954509]
 [ 9.51066022  8.30228618 10.07105129  9.56380283]
 [ 8.66962459  7.20964881  8.69532893  8.82893475]
 [ 6.82071329  5.98291558  7.25754973  6.83224348]
 [ 9.28152279  8.09240248  9.53421533  9.26935192]
 [ 8.27133242  7.20598577  8.48101371  8.26637347]
 [ 8.57657662  7.39740925  8.7493711   8.59706821]
 [ 7.54030718  6.79539344  8.30665813  7.49834759]
 [ 8.85486768  8.04356926  9.33640486  8.72194919]
 [ 9.44172624  8.2789123  10.32095247  9.50041196]
 [ 9.35172245  8.34003298  9.5779102   9.22668215]
 [ 6.5815038   5.82818468  7.02997988  6.54115409]
 [ 9.76042623  8.71640443 10.19552607  9.64212063]
 [ 9.70577473  8.43378806 10.281715    9.76048996]
 [ 8.35286468  7.33903999  8.58699492  8.30783715]
 [ 7.3017924   6.13495191  7.03903967  7.36510821]
 [ 8.432473    7.25127987  8.65664861  8.50173565]
 [ 8.10863902  6.90931824  8.56

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [48]:
user_mat, movie_mat = FunkSVD(user_movie_subset, latent_features=4, learning_rate=0.005, iters=250)
#use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 45.105339
2 		 16.435957
3 		 4.513333
4 		 2.900157
5 		 2.749862
6 		 2.714609
7 		 2.692598
8 		 2.671361
9 		 2.648541
10 		 2.623492
11 		 2.595770
12 		 2.564886
13 		 2.530302
14 		 2.491450
15 		 2.447763
16 		 2.398700
17 		 2.343774
18 		 2.282592
19 		 2.214895
20 		 2.140604
21 		 2.059871
22 		 1.973120
23 		 1.881072
24 		 1.784758
25 		 1.685490
26 		 1.584804
27 		 1.484371
28 		 1.385879
29 		 1.290909
30 		 1.200823
31 		 1.116669
32 		 1.039137
33 		 0.968553
34 		 0.904908
35 		 0.847917
36 		 0.797093
37 		 0.751812
38 		 0.711382
39 		 0.675090
40 		 0.642242
41 		 0.612185
42 		 0.584329
43 		 0.558150
44 		 0.533194
45 		 0.509080
46 		 0.485496
47 		 0.462197
48 		 0.439002
49 		 0.415790
50 		 0.392495
51 		 0.369105
52 		 0.345653
53 		 0.322216
54 		 0.298904
55 		 0.275854
56 		 0.253224
57 		 0.231178
58 		 0.209882
59 		 0.189495
60 		 0.170157
61 		 0.151984
62 		 0.135065
63 		 0.119458
64 		

In [49]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[10.0000081  10.00000491 10.0000022   9.99999086]
 [10.00000135  4.00000071  9.00000026  9.99999845]
 [ 7.99999896  8.99999938  9.99999967  5.00000101]
 [ 9.00000254  8.00000142 10.00000056  9.99999708]
 [ 9.99999611  4.9999976   8.99999885  9.00000425]
 [ 5.99999873  3.99999928  9.99999966  6.00000132]
 [ 8.99999966  7.99999978  9.99999987  9.00000029]
 [10.00000491  5.00000296  9.00000129  7.99999442]
 [ 7.00000767  8.0000045  10.00000193  7.99999134]
 [ 9.00000135  5.00000064  9.00000018  6.99999842]
 [ 9.00000012  7.99999992  9.99999982  7.99999972]
 [ 9.00000587 10.00000332 10.00000131  8.99999329]
 [ 9.99999647  8.99999764  9.99999876  8.00000379]
 [ 4.9999977   7.99999844  4.99999918  8.00000249]
 [10.00000078  8.00000033 10.          9.99999899]
 [ 8.99999432  8.99999647  9.9999983  10.00000621]
 [ 8.99999372  7.99999624  7.99999829  8.00000691]
 [10.00000062  8.00000043  1.0000002   9.99999926]
 [ 4.9999982   5.99999895  9.99999952 10.00000193]
 [ 7.99999954  6.99999985  9.99

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [50]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

In [51]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(user_movie_subset, latent_features=4, learning_rate=0.005, iters=250)
#use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 44.086576
2 		 15.818621
3 		 4.372445
4 		 2.880368
5 		 2.734493
6 		 2.687619
7 		 2.651082
8 		 2.613824
9 		 2.572982
10 		 2.527187
11 		 2.475385
12 		 2.416639
13 		 2.350137
14 		 2.275255
15 		 2.191656
16 		 2.099388
17 		 1.998984
18 		 1.891532
19 		 1.778684
20 		 1.662603
21 		 1.545799
22 		 1.430909
23 		 1.320425
24 		 1.216444
25 		 1.120483
26 		 1.033409
27 		 0.955468
28 		 0.886409
29 		 0.825634
30 		 0.772356
31 		 0.725721
32 		 0.684888
33 		 0.649076
34 		 0.617576
35 		 0.589756
36 		 0.565050
37 		 0.542950
38 		 0.523002
39 		 0.504795
40 		 0.487963
41 		 0.472181
42 		 0.457161
43 		 0.442654
44 		 0.428444
45 		 0.414351
46 		 0.400227
47 		 0.385952
48 		 0.371438
49 		 0.356623
50 		 0.341471
51 		 0.325974
52 		 0.310147
53 		 0.294027
54 		 0.277673
55 		 0.261162
56 		 0.244588
57 		 0.228055
58 		 0.211677
59 		 0.195572
60 		 0.179854
61 		 0.164637
62 		 0.150022
63 		 0.136099
64 		

In [52]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 10.000000937787139:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [56]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20)
#fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 23.232588
2 		 10.719308
3 		 7.386896
4 		 5.704763
5 		 4.639247
6 		 3.891290
7 		 3.333424
8 		 2.900712
9 		 2.555776
10 		 2.275258
11 		 2.043733
12 		 1.850566
13 		 1.688115
14 		 1.550655
15 		 1.433731
16 		 1.333768
17 		 1.247844
18 		 1.173551
19 		 1.108903
20 		 1.052270


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [58]:
# Replace each of the comments below with the correct values
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))# How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings# How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [59]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
